In [65]:
# importing libraries
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tqdm import tqdm
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\80316\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Function-1: Prediction

In [51]:
def final_fun_1(data):
    if 'label' in data.columns:
        data.drop('label', axis=1, inplace=True)
    data.drop(['date', 'created_utc'], axis=1, inplace=True)
    
    stop_words = stopwords.words('english')

    def decontracted(phrase):
        # specific
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

        # https://gist.github.com/sebleier/554280

        # Remove Emoji
    def deEmojify(text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)

    def preprocess_text(text_data):
        preprocessed_text = []
        # tqdm is for printing the status bar
        for sentance in tqdm(text_data):
            sent = decontracted(sentance)
            sent=deEmojify(sent)
            sent = sent.replace('\\r', ' ')
            sent = sent.replace('\\n', ' ')
            sent = sent.replace('\\"', ' ')
            sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
            # https://gist.github.com/sebleier/554280
            sent = ' '.join(e for e in sent.split() if e not in stop_words)
            preprocessed_text.append(sent.lower().strip())
        return preprocessed_text
    
    data['comment'] = preprocess_text(data['comment'].values)
    data['parent_comment'] = preprocess_text(data['parent_comment'].values)
        
    data['subreddit'] = data['subreddit'].str.replace(' ','')
    data['subreddit'] = data['subreddit'].str.replace('-','_')
    data['subreddit'] = data['subreddit'].str.lower()
    
    data['author'] = data['author'].str.replace(' ','')
    data['author'] = data['author'].str.replace('-','_')
    data['author'] = data['author'].str.lower()
    
    neg_comment = []
    neu_comment = []
    pos_comment = []
    comp_comment = []

    sid = SentimentIntensityAnalyzer()

    for sentence in tqdm(data['comment']):
        ss = sid.polarity_scores(sentence)
        neg_comment.append(ss['neg'])
        neu_comment.append(ss['neu'])
        pos_comment.append(ss['pos'])
        comp_comment.append(ss['compound'])

    data['Neg_comment'] = neg_comment
    data['Neu_comment'] = neu_comment
    data['Pos_comment'] = pos_comment
    data['Compound_comment'] = comp_comment
    
    neg_pcomment = []
    neu_pcomment = []
    pos_pcomment = []
    comp_pcomment = []

    sid = SentimentIntensityAnalyzer()

    for sentence in tqdm(data['parent_comment']):
        ss = sid.polarity_scores(sentence)
        neg_pcomment.append(ss['neg'])
        neu_pcomment.append(ss['neu'])
        pos_pcomment.append(ss['pos'])
        comp_pcomment.append(ss['compound'])
    
    data['Neg_pcomment'] = neg_pcomment
    data['Neu_pcomment'] = neu_pcomment
    data['Pos_pcomment'] = pos_pcomment
    data['Compound_pcomment'] = comp_pcomment
    
    data['comment_len']=data['comment'].apply(lambda x:len(x.split()))
    data['pcomment_len']=data['parent_comment'].apply(lambda x:len(x.split()))
    
    cat_author, cat_subreddit = pickle.load(open("cat.pkl", 'rb'))
    data['author'] = cat_author.transform(data['author'])
    data['subreddit'] = cat_subreddit.transform(data['subreddit'])
    
    scaler_score, scaler_ups, scaler_negc, scaler_neuc, scaler_posc, scaler_compc, scaler_negpc, scaler_neupc, scaler_pospc, scaler_comppc, scaler_comlen, scaler_pcomlen = pickle.load(open("scale.pkl", 'rb'))
    data['score'] = scaler_score.transform(data['score'].values.reshape(-1,1))    
    data['ups'] = scaler_ups.transform(data['ups'].values.reshape(-1,1))
    data['Neg_comment'] = scaler_negc.transform(data['Neg_comment'].values.reshape(-1,1))
    data['Neu_comment'] = scaler_neuc.transform(data['Neu_comment'].values.reshape(-1,1))
    data['Pos_comment'] = scaler_posc.transform(data['Pos_comment'].values.reshape(-1,1))
    data['Compound_comment'] = scaler_compc.transform(data['Compound_comment'].values.reshape(-1,1))
    data['Neg_pcomment'] = scaler_negpc.transform(data['Neg_pcomment'].values.reshape(-1,1))
    data['Neu_pcomment'] = scaler_neupc.transform(data['Neu_pcomment'].values.reshape(-1,1))
    data['Pos_pcomment'] = scaler_pospc.transform(data['Pos_pcomment'].values.reshape(-1,1))
    data['Compound_pcomment'] = scaler_comppc.transform(data['Compound_pcomment'].values.reshape(-1,1))
    data['comment_len'] = scaler_comlen.transform(data['comment_len'].values.reshape(-1,1))
    data['pcomment_len'] = scaler_pcomlen.transform(data['pcomment_len'].values.reshape(-1,1))
    
    text_features = data[['comment', 'parent_comment']]
    data.drop(['comment', 'parent_comment'], axis=1, inplace=True)
    
    token_comment, token_pcomment =  pickle.load(open('tokens.pkl','rb'))
    sequence_com = token_comment.texts_to_sequences(text_features['comment'])
    pad_comment = pad_sequences(sequence_com, maxlen=50, padding='post')
    sequence_pcom = token_pcomment.texts_to_sequences(text_features['parent_comment'])
    pad_pcomment = pad_sequences(sequence_pcom, maxlen=500, padding='post')
    
    num_data = data.values
    
    model = keras.models.load_model('final_model.h5')
        
    predictions = model.predict([pad_comment, pad_pcomment, num_data])
    
    prediction_list = []
    
    for prediction in predictions:
        if prediction >= 0.5:
            prediction_list.append(1)
            prediction = 'Sarcastic'
            return prediction
        else:
            prediction_list.append(0)
            prediction = 'Non-Sarcastic'
            return prediction

In [52]:
df = pd.read_csv('train-balanced-sarcasm.csv')
sample_data = df.sample(1)
sample_data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
204933,1,Yellow snow flavored :D,Vapeisntsobad,electronic_cigarette,0,0,0,2016-05,2016-05-23 00:53:12,What flavor juice do you wish existed but you ...


In [53]:
final_fun_1(sample_data)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 144.43it/s]


'Non-Sarcastic'

In [61]:
def final_fun_1(data, y_true):
    if 'label' in data.columns:
        data.drop('label', axis=1, inplace=True)
    data.drop(['date', 'created_utc'], axis=1, inplace=True)
    
    stop_words = stopwords.words('english')

    def decontracted(phrase):
        # specific
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

        # https://gist.github.com/sebleier/554280

        # Remove Emoji
    def deEmojify(text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)

    def preprocess_text(text_data):
        preprocessed_text = []
        # tqdm is for printing the status bar
        for sentance in tqdm(text_data):
            sent = decontracted(sentance)
            sent=deEmojify(sent)
            sent = sent.replace('\\r', ' ')
            sent = sent.replace('\\n', ' ')
            sent = sent.replace('\\"', ' ')
            sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
            # https://gist.github.com/sebleier/554280
            sent = ' '.join(e for e in sent.split() if e not in stop_words)
            preprocessed_text.append(sent.lower().strip())
        return preprocessed_text
    
    data['comment'] = preprocess_text(data['comment'].values)
    data['parent_comment'] = preprocess_text(data['parent_comment'].values)
        
    data['subreddit'] = data['subreddit'].str.replace(' ','')
    data['subreddit'] = data['subreddit'].str.replace('-','_')
    data['subreddit'] = data['subreddit'].str.lower()
    
    data['author'] = data['author'].str.replace(' ','')
    data['author'] = data['author'].str.replace('-','_')
    data['author'] = data['author'].str.lower()
    
    neg_comment = []
    neu_comment = []
    pos_comment = []
    comp_comment = []

    sid = SentimentIntensityAnalyzer()

    for sentence in tqdm(data['comment']):
        ss = sid.polarity_scores(sentence)
        neg_comment.append(ss['neg'])
        neu_comment.append(ss['neu'])
        pos_comment.append(ss['pos'])
        comp_comment.append(ss['compound'])

    data['Neg_comment'] = neg_comment
    data['Neu_comment'] = neu_comment
    data['Pos_comment'] = pos_comment
    data['Compound_comment'] = comp_comment
    
    neg_pcomment = []
    neu_pcomment = []
    pos_pcomment = []
    comp_pcomment = []

    sid = SentimentIntensityAnalyzer()

    for sentence in tqdm(data['parent_comment']):
        ss = sid.polarity_scores(sentence)
        neg_pcomment.append(ss['neg'])
        neu_pcomment.append(ss['neu'])
        pos_pcomment.append(ss['pos'])
        comp_pcomment.append(ss['compound'])
    
    data['Neg_pcomment'] = neg_pcomment
    data['Neu_pcomment'] = neu_pcomment
    data['Pos_pcomment'] = pos_pcomment
    data['Compound_pcomment'] = comp_pcomment
    
    data['comment_len']=data['comment'].apply(lambda x:len(x.split()))
    data['pcomment_len']=data['parent_comment'].apply(lambda x:len(x.split()))
    
    cat_author, cat_subreddit = pickle.load(open("cat.pkl", 'rb'))
    data['author'] = cat_author.transform(data['author'])
    data['subreddit'] = cat_subreddit.transform(data['subreddit'])
    
    scaler_score, scaler_ups, scaler_negc, scaler_neuc, scaler_posc, scaler_compc, scaler_negpc, scaler_neupc, scaler_pospc, scaler_comppc, scaler_comlen, scaler_pcomlen = pickle.load(open("scale.pkl", 'rb'))
    data['score'] = scaler_score.transform(data['score'].values.reshape(-1,1))    
    data['ups'] = scaler_ups.transform(data['ups'].values.reshape(-1,1))
    data['Neg_comment'] = scaler_negc.transform(data['Neg_comment'].values.reshape(-1,1))
    data['Neu_comment'] = scaler_neuc.transform(data['Neu_comment'].values.reshape(-1,1))
    data['Pos_comment'] = scaler_posc.transform(data['Pos_comment'].values.reshape(-1,1))
    data['Compound_comment'] = scaler_compc.transform(data['Compound_comment'].values.reshape(-1,1))
    data['Neg_pcomment'] = scaler_negpc.transform(data['Neg_pcomment'].values.reshape(-1,1))
    data['Neu_pcomment'] = scaler_neupc.transform(data['Neu_pcomment'].values.reshape(-1,1))
    data['Pos_pcomment'] = scaler_pospc.transform(data['Pos_pcomment'].values.reshape(-1,1))
    data['Compound_pcomment'] = scaler_comppc.transform(data['Compound_pcomment'].values.reshape(-1,1))
    data['comment_len'] = scaler_comlen.transform(data['comment_len'].values.reshape(-1,1))
    data['pcomment_len'] = scaler_pcomlen.transform(data['pcomment_len'].values.reshape(-1,1))
    
    text_features = data[['comment', 'parent_comment']]
    data.drop(['comment', 'parent_comment'], axis=1, inplace=True)
    
    token_comment, token_pcomment =  pickle.load(open('tokens.pkl','rb'))
    sequence_com = token_comment.texts_to_sequences(text_features['comment'])
    pad_comment = pad_sequences(sequence_com, maxlen=50, padding='post')
    sequence_pcom = token_pcomment.texts_to_sequences(text_features['parent_comment'])
    pad_pcomment = pad_sequences(sequence_pcom, maxlen=500, padding='post')
    
    num_data = data.values
    
    model = keras.models.load_model('final_model.h5')
        
    predictions = model.predict([pad_comment, pad_pcomment, num_data])
    
    prediction_list = []
    
    for prediction in predictions:
        if prediction >= 0.5:
            prediction_list.append(1)
            
        else:
            prediction_list.append(0)
            
    y_pred = np.array(prediction_list)

    accuracy = accuracy_score(y_true, y_pred)
    
    return accuracy

In [58]:
df = pd.read_csv('train-balanced-sarcasm.csv')

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
776248,0,There is a difference when you bring something...,ZaynRocks,SquaredCircle,3,3,0,2015-03,2015-03-07 19:15:29,There's a difference when you criticize the pr...


In [65]:
sample_data = df.sample(50)
sample_data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
913917,1,But but.. How are those stupid Christians gonn...,MNREDR,cringepics,1,1,0,2013-04,2013-04-16 18:48:14,I would like to ask my fellow atheists to lear...
931108,1,Further evidence that there are likely too man...,zxcvb94105,Green,1,1,0,2013-06,2013-06-16 06:25:45,Plant erupts in flames while chemical industry...
208340,1,Well technically reddit is a private interface...,TheBuffaloSabres,The_Donald,1,1,0,2016-07,2016-07-03 00:51:20,"Yeah, I've been told that too, I've been banne..."
624824,0,How does it work exactly?,AbusedToasters,mildlyinteresting,1,1,0,2015-08,2015-08-19 22:07:54,Infinity Monkey Bar
956517,0,"Road bike sub $400, I would look on craigslist...",aggieotis,bicycling,1,1,0,2012-05,2012-05-30 19:43:09,What bikes would you recommend?


In [66]:
final_fun_1(sample_data, sample_data['label'])

100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 1372.44it/s]


0.8

In [66]:
df = pd.read_csv('train-balanced-sarcasm.csv')

In [71]:
sample_data = df.sample(1)
sample_data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
781858,1,you should file a dmca complaint,BrokenStool,GlobalOffensive,1,1,0,2014-12,2014-12-16 05:43:57,This skin looks exactly like the SA Highwayman...


In [73]:
sample_data['parent_comment'].values[0]

'This skin looks exactly like the SA Highwayman...Did they steal the art? Or did they just change the name with permission?'

In [76]:
sample_data['comment'].values[0]

'you should file a dmca complaint'

In [75]:
sample_data['author'].values[0]

'BrokenStool'

In [74]:
sample_data['subreddit'].values[0]

'GlobalOffensive'